Reflection and Blogpost Writing

## Setup

In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

Title: "Demystifying DeepLearning.AI: Your Gateway to AI Excellence"

Dive into the world of artificial intelligence with DeepLearning.AI! Founded by the renowned Andrew Ng, this platform offers top-tier courses to help you grasp the complexities of deep learning. Whether you're a novice or an expert, the structured curriculum caters to all levels. From computer vision to natural language processing, the program covers it all. Gain practical skills, work on hands-on projects, and join a thriving community of learners worldwide. Embrace the future of technology with DeepLearning.AI as your guiding light. Start your AI journey today!


## Adding reflection 

A critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Demystifying DeepLearning.AI: Your Gateway to AI Excellence"

Dive into the world of artificial intelligence with DeepLearning.AI! Founded by the renowned Andrew Ng, this platform offers top-tier courses to help you grasp the complexities of deep learning. Whether you're a novice or an expert, the structured curriculum caters to all levels. From computer vision to natural language processing, the program covers it all. Gain practical skills, work on hands-on projects, and join a thriving community of learners worldwide. Embrace the future of technology with DeepLearning.AI as your guiding light. Start your AI journey today!

--------------------------------------------------------------------------------
Critic (to Writer)

## Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Demystifying DeepLearning.AI: Your Gateway to AI Excellence"

Dive into the world of artificial intelligence with DeepLearning.AI! Founded by the renowned Andrew Ng, this platform offers top-tier courses to help you grasp the complexities of deep learning. Whether you're a novice or an expert, the structured curriculum caters to all levels. From computer vision to natural language processing, the program covers it all. Gain practical skills, work on hands-on projects, and join a thriving community of learners worldwide. Embrace the future of technology with DeepLearning.AI as your guiding light. Start your AI journey today!

--------------------------------------------------------------------------------

*****************


--------------------------------------------------------------------------------
Meta Reviewer (to Critic):

Aggregating the feedback from the reviewers, it is clear that there are some key areas to address in the content:

1. **Title, Keyword Integration, and Call-to-Action:** The SEO Reviewer has suggested optimizing the content in these areas, which is important for visibility and engagement.

2. **Claim Verification:** Both the Legal and Ethics Reviewers have highlighted the need to ensure that the claims made in the content, such as "top-tier courses," "AI excellence," and terms like "renowned" and "thriving community," are substantiated with evidence. This is crucial to avoid misleading representations and potential legal issues related to false advertising.

Final Suggestions:
- **Evidence-based Claims:** It is essential to provide evidence or context to support any claims made about the courses or community to maintain credibility and transparency.
- **Avoid Subjective Languag

## Get the summary

In [15]:
print(res.summary)

Title: "Unveiling DeepLearning.AI: Accelerate Your AI Journey"

Embark on your AI learning path with DeepLearning.AI, the brainchild of the esteemed Andrew Ng. This platform offers a comprehensive range of courses tailored for all skill levels, covering diverse fields like computer vision and natural language processing. Engage with practical projects, refine your skills, and collaborate with a global community of learners. It's vital to substantiate claims of "top-tier courses" and a "thriving community" with verifiable evidence. By providing transparent, evidence-based information and maintaining legal and ethical standards, DeepLearning.AI stands as a beacon for AI enthusiasts worldwide. Start your AI journey today!
